In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qadence import *
import torch
from torch.autograd import grad

In [2]:
def generate_chebyshev_grid(n, a=0, b=0.9, requires_grad=True):
    k = torch.arange(n)
    chebyshev_nodes = torch.cos(torch.pi * (2*k + 1) / (2*n))  #standard Chebyshev nodes in [-1,1]
    scaled_nodes = ((chebyshev_nodes + 1) / 2) * (b - a) + a  #rescale to [a, b]
    
    return scaled_nodes.requires_grad_(requires_grad=requires_grad)

N_POINTS = 10
x = generate_chebyshev_grid(N_POINTS)
print(x)
cp = torch.reshape(x, (N_POINTS,1))

tensor([0.8945, 0.8510, 0.7682, 0.6543, 0.5204, 0.3796, 0.2457, 0.1318, 0.0490,
        0.0055], requires_grad=True)


In [3]:
#N_POINTS = 20
#cp = torch.linspace(0.0, 0.9, N_POINTS, dtype=torch.float64)
#cp = torch.reshape(cp, (N_POINTS,1))
#print(cp)

### f(x)

In [4]:
N_QUBITS = 6
DEPTH = 2

torch.manual_seed(42)
ansatz = hea(n_qubits=N_QUBITS, depth=DEPTH, operations=[RZ, RX, RZ])

chebT = feature_map(n_qubits=N_QUBITS, param="x", op=RY, fm_type = BasisSet.CHEBYSHEV, reupload_scaling = ReuploadScaling.TOWER)
fm = chebT

obs = total_magnetization(N_QUBITS)
circuit = QuantumCircuit(N_QUBITS, chain(fm, ansatz))
model = QNN(circuit=circuit, observable=obs, inputs=["x"])

inital_vparams = model.vparams
#print("Inital: ",inital_vparams)
sorted_keys = sorted(inital_vparams.keys(), key=lambda k: int(k.split('_')[1]))
inital_thetas = np.array([inital_vparams[k].item() for k in sorted_keys])
inital_thetas = np.reshape(inital_thetas, (DEPTH, 3, N_QUBITS))
Theta_init = np.transpose(inital_thetas, (2, 0, 1))
theta_init = Theta_init.reshape(-1)
print("For Pennylane: ",theta_init)
#check
#Theta_init = np.reshape(theta_init, (N_QUBITS,DEPTH,3))
#print("Theta: ",Theta_init)


print("Result: ",model(cp[9]))
print(model)

For Pennylane:  [0.05815449 0.9552357  0.84251973 0.62340308 0.03134017 0.25185522
 0.06291017 0.92875258 0.87050104 0.437279   0.04036647 0.23340819
 0.12358607 0.08354337 0.37753296 0.07473488 0.93190879 0.2256651
 0.05258016 0.13264067 0.61243659 0.68343022 0.15212583 0.3175678
 0.5261719  0.1570535  0.08810978 0.31223868 0.26504491 0.61440828
 0.47678475 0.37536703 0.70118189 0.30905878 0.13043349 0.59777135]
Result:  tensor([[-0.5827]], grad_fn=<CatBackward0>)
QNN(
  (_circuit): ConvertedCircuit(
    (native): QuantumCircuit(
      (operations): ModuleList(
        (0): Sequence(
          (operations): ModuleList(
            (0): Sequence(
              (operations): ModuleList(
                (0): RY(target: (0,), param: 1_0*acos(x))
                (1): RY(target: (1,), param: 2_0*acos(x))
                (2): RY(target: (2,), param: 3_0*acos(x))
                (3): RY(target: (3,), param: 4_0*acos(x))
                (4): RY(target: (4,), param: 5_0*acos(x))
               